In [3]:
import glob
import os


In [4]:
def find_files_with_prefix(directory, prefix):
    pattern = os.path.join(directory, prefix + '*')
    matching_files = glob.glob(pattern)
    
    # Sort the matching files alphabetically
    matching_files.sort()
    
    return matching_files

In [5]:
directory_path = graph_dir
prefix = 'Left_B_02_034-TRY-0LGN-101'

result = find_files_with_prefix(directory_path, prefix)
print(result)

['../src/graph\\Left_B_02_034-TRY-0LGN-101_000000.json', '../src/graph\\Left_B_02_034-TRY-0LGN-101_000001.json', '../src/graph\\Left_B_02_034-TRY-0LGN-101_000002.json', '../src/graph\\Left_B_02_034-TRY-0LGN-101_000003.json', '../src/graph\\Left_B_02_034-TRY-0LGN-101_000004.json', '../src/graph\\Left_B_02_034-TRY-0LGN-101_000005.json', '../src/graph\\Left_B_02_034-TRY-0LGN-101_000006.json', '../src/graph\\Left_B_02_034-TRY-0LGN-101_000007.json', '../src/graph\\Left_B_02_034-TRY-0LGN-101_000008.json', '../src/graph\\Left_B_02_034-TRY-0LGN-101_000009.json', '../src/graph\\Left_B_02_034-TRY-0LGN-101_000010.json', '../src/graph\\Left_B_02_034-TRY-0LGN-101_000011.json', '../src/graph\\Left_B_02_034-TRY-0LGN-101_000012.json', '../src/graph\\Left_B_02_034-TRY-0LGN-101_000013.json', '../src/graph\\Left_B_02_034-TRY-0LGN-101_000014.json', '../src/graph\\Left_B_02_034-TRY-0LGN-101_000015.json', '../src/graph\\Left_B_02_034-TRY-0LGN-101_000016.json', '../src/graph\\Left_B_02_034-TRY-0LGN-101_00001

In [1]:
from graph_tools import GraphTool
from paths import *

In [3]:
gt = GraphTool(graph_dir, "Left_B_02_034-TRY-0LGN-101")

In [6]:
gt.features.shape

(95, 21, 3)

In [1]:
import numpy as np
from configs import *

In [2]:
window_flags = np.array([0, 0, 0, 1, 1, 1])

In [3]:
ok_indices = np.where(window_flags == FLAG_OK)[0]

In [4]:
ok_indices

array([3, 4, 5], dtype=int64)

In [6]:
window_flags == FLAG_OK

array([False, False, False,  True,  True,  True])

In [8]:
np.where(window_flags == FLAG_OK)[0]

array([3, 4, 5], dtype=int64)

In [10]:
new_flags = np.array([0, 0, 0, 0, 0])

In [12]:
np.where(new_flags == FLAG_OK)[0]

array([], dtype=int64)

In [11]:
np.min(np.where(new_flags == FLAG_OK)[0])

ValueError: zero-size array to reduction operation minimum which has no identity

In [3]:
import numpy as np

In [4]:
def find_closest_index_with_value(arr, target_index, target_value):
    # Find the indices of elements with the target value
    indices_with_target_value = np.where(arr == target_value)[0]

    # Calculate the absolute differences between these indices and the target index
    differences = np.abs(indices_with_target_value - target_index)

    # Find the index with the smallest difference
    closest_index = indices_with_target_value[np.argmin(differences)]

    return closest_index

In [10]:
# Example usage
arr = np.array([3, 7, 2, 3, 5])
target_index = 1  # Index of the current focus
target_value = 6   # Value to search for

closest_index = find_closest_index_with_value(arr, target_index, target_value)
print("Closest index with the target value:", closest_index)
print("Closest element with the target value:", arr[closest_index])


ValueError: attempt to get argmin of an empty sequence